In [173]:
import pandas as pd
import os
import csv
import numpy as np

In [35]:
counties_2001 = pd.read_csv('county_data/' + 'Underlying Cause of Death, 1999-2017_2012_counties.txt', sep='\t')
counties_2002 = pd.read_csv('county_data/' + 'Underlying Cause of Death, 1999-2017_2002_counties.txt', sep='\t')

In [79]:
counties_data = pd.read_csv('county_data/' + 'Underlying Cause of Death, 1999-2017_2001_counties.txt', sep='\t')

for file in os.listdir('county_data'):
    if file != 'Underlying Cause of Death, 1999-2017_2012_counties.txt':
        new_year = pd.read_csv('county_data/' + file, sep='\t', na_filter = False)
        counties_data = pd.concat([counties_data, new_year], axis=0)


/Users/Marta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [80]:
col_list = ['Notes', 'Population', 'State', 'State Code', 'Crude Rate']
for name in col_list:
    counties_data = counties_data.drop(columns = name)



In [83]:
new = counties_data["County"].str.split(",", n = 1, expand = True)

In [87]:
counties_data["County"]= new[0]
counties_data['State'] = new[1]

In [93]:
new_date = counties_data['Month Code'].str.split('/', n=1, expand=True)

In [94]:
counties_data['Year']=new_date[0]
counties_data['Month']=new_date[1]

In [98]:
state_data = pd.read_csv('Underlying Cause of Death, 1999-2017_allyears_states.txt', sep='\t')

In [118]:
#Dataframe of state populations from 2000-2010
state_pops_2000 = pd.read_csv('co-est00int-tot.csv', encoding='latin-1')

In [ ]:
#Dataframe of state populations from 2010-2018
state_pops_2010 = pd.read_csv('co-est2018-alldata.csv', encoding='latin-1')

In [138]:
#Making a dataframe of the states and their abbreviations
state_abrevs = pd.read_csv('state_abrevs.csv')

In [141]:
#Renaming columns in the state abbrevs dataframe so that they match existing column names in the previous dataframes
state_abrevs = state_abrevs.rename(columns={"State": "STNAME", 'State Abrev': 'State'})

In [155]:
#Merging the state abrevs with the county population data for 2010
state_pops_2010_mergetest = pd.merge(state_pops_2010[['STNAME', 'CTYNAME', 'CENSUS2010POP',
       'ESTIMATESBASE2010', 'POPESTIMATE2010', 'POPESTIMATE2011',
       'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014',
       'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017',
       'POPESTIMATE2018']], state_abrevs, on='STNAME')

In [157]:
#Changing the name of the CTYNAME column in the population data 2010
state_pops_2010_mergetest = state_pops_2010_mergetest.rename(columns={'CTYNAME':'County'})

In [167]:
#Merging county population data for 2010 with county death rates
county_pop_merge = pd.merge(state_pops_2010_mergetest, counties_data, on=['County'])


In [181]:
#Checking to see if it matched the wrong county names anywhere
#Need to work on this because right now it's not recognizing that the two columns have the same values in them
county_pop_merge['que'] = np.where((county_pop_merge['State_x']==county_pop_merge['State_y']), county_pop_merge['State_y'], np.nan)

In [197]:
#Choosing values only for a specific year
county_pop_2010 = county_pop_merge[['STNAME', 'County',
       'POPESTIMATE2010', 'State_x',
       '% of Total Deaths', 'County Code', 'Deaths', 'Month', 'Month Code',
       'State_y', 'Year', 'que']].loc[county_pop_merge["Year"]=='2010']

In [198]:
#Renaming the population column
county_pop_2010 = county_pop_2010.rename(columns={'POPESTIMATE2010':'Population'})

In [202]:
county_pop_2010

,STNAME,County,Population,State_x,% of Total Deaths,County Code,Deaths,Month,Month Code,State_y,Year,que
134,Alabama,Baldwin County,183111,AL,NaN,01003,15,01,2010/01,AL,2010,AL
135,Alabama,Baldwin County,183111,AL,NaN,01003,23,02,2010/02,AL,2010,AL
136,Alabama,Baldwin County,183111,AL,NaN,01003,14,03,2010/03,AL,2010,AL
137,Alabama,Baldwin County,183111,AL,NaN,01003,14,04,2010/04,AL,2010,AL
138,Alabama,Baldwin County,183111,AL,NaN,01003,24,05,2010/05,AL,2010,AL
139,Alabama,Baldwin County,183111,AL,NaN,01003,13,06,2010/06,AL,2010,AL
140,Alabama,Baldwin County,183111,AL,NaN,01003,17,09,2010/09,AL,2010,AL
141,Alabama,Baldwin County,183111,AL,NaN,01003,13,10,2010/10,AL,2010,AL
142,Alabama,Baldwin County,183111,AL,NaN,01003,20,12,2010/12,AL,2010,AL
283,Georgia,Baldwin County,45701,GA,NaN,01003,15,01,2010/01,AL,2010,AL


In [108]:
#splitting the county name from the word county
new_county_name = state_pops_2000['CTYNAME'].str.split(' ', n=1, expand=True)
state_pops_2000['County'] = new_county_name[0]

In [111]:
#splitting the county name from the word county
new_county_name = state_pops_2010['CTYNAME'].str.split(' ', n=1, expand=True)
state_pops_2010['County'] = new_county_name[0]

In [113]:
counties_data

,% of Total Deaths,County,County Code,Deaths,Month,Month Code,State,Year
0,0.0%,Baldwin County,1003,13,01,2001/01,AL,2001
1,0.0%,Baldwin County,1003,14,02,2001/02,AL,2001
2,0.0%,Baldwin County,1003,17,03,2001/03,AL,2001
3,0.0%,Baldwin County,1003,12,09,2001/09,AL,2001
4,0.0%,Baldwin County,1003,14,10,2001/10,AL,2001
5,0.0%,Blount County,1009,10,01,2001/01,AL,2001
6,0.0%,Calhoun County,1015,16,01,2001/01,AL,2001
7,0.0%,Calhoun County,1015,13,02,2001/02,AL,2001
8,0.0%,Calhoun County,1015,14,03,2001/03,AL,2001
9,0.0%,Calhoun County,1015,11,04,2001/04,AL,2001
